In [4]:
import subprocess
from multiprocessing import Pool,cpu_count
import os
from itertools import product,repeat
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.interpolate import spline
from scipy.ndimage.filters import gaussian_filter1d
from scipy import stats
import fileinput as fi
import pickle as pkl


%run averaging.ipynb
%run optimize_slope.ipynb

ncores=cpu_count()

ERROR:root:File `'averaging.ipynb.py'` not found.
ERROR:root:File `'optimize_slope.ipynb.py'` not found.


In [5]:
area_er = 2*(3.9+0.1+0.1*0.1+3.9*0.1)
vol_er = (3.9*0.1*0.1)
vol_cyt = (4.0*0.5*0.5-vol_er)
vol_tot = vol_cyt + vol_er
N_avo=6.0221409e23

# Search and Replace in files

In [4]:
dataloc="/home/kabir/Project/tripartiteSynapse/mcell/R150control_clamped/"
files=os.listdir(dataloc)
RS_files=[]
for f in files:
    if "RSI" in f:
        RS_files.append(f)
#RSI_files

In [14]:
for fn in RS_files:
    with fi.FileInput(dataloc+fn,inplace=True, backup='.bak') as f:
        for line in f:
            print(line.replace('"ppf/','"ppf/clamped/'), end='')

In [7]:
for fn in RS_files:
    with fi.FileInput(dataloc+fn,inplace=True, backup='.bak') as f:
        for line in f:
            print(line.replace('"RS20p"','"freq/no_ryr"'), end='')

In [7]:
p=[-1.205e-06,  4.109e-04, -3.469e-02,  8.770e-01] #polynomial from tools/RSI_script_VDCC-clamped.ipynb
loc="/home/kabir/Project/tripartiteSynapse/murthy/Pr distribution-Murthy1997.dat"

dist=np.loadtxt(loc)

vdcc_numbers=np.zeros(len(dist.T[0]))
vdcc_dist={}

for i,d in enumerate(dist.T[0]):
    P=p[:3]+[p[3]-d]
    #print(P)
    roots=np.roots(P)
    real_roots=np.real(roots[np.isreal(roots)])
    #print(real_roots)
    for r in real_roots:
        if r>=60 and r<=160:
            vdcc_number=int(r)
            vdcc_numbers[i]=vdcc_number
            #print(i)
            vdcc_dist[vdcc_number]=int(dist.T[1][i]/sum(dist.T[1])*2000)

vdcc_numbers,vdcc_dist

(array([ 70.,  78.,  84.,  89.,  94.,  98., 103., 107., 111., 115., 120.,
        124., 128., 133., 138., 143., 149., 157.,   0.,   0.]),
 {70: 107,
  78: 329,
  84: 391,
  89: 326,
  94: 274,
  98: 166,
  103: 120,
  107: 78,
  111: 58,
  115: 48,
  120: 22,
  124: 29,
  128: 6,
  133: 6,
  138: 13,
  143: 3,
  149: 9,
  157: 0})

In [6]:
sims=["stores_blocked","R150control_clamped","R300ER2x_clamped" ,"R300ER3x_clamped","R150ER3x_clamped" ,"R150ER2x_clamped"]
isi_range=range(20,201,40)
for sim in sims:
    for isi in isi_range:
        #isi=100
        temp_loc="/home/kabir/Project/tripartiteSynapse/mcell/"+sim+"/RSI"+str(isi)+"V80.mdl"
        with open(temp_loc,'r') as template:
            template_lines = template.readlines()
        for vdcc_number in vdcc_numbers:
            new_loc="/home/kabir/Project/tripartiteSynapse/mcell/"+sim+"/RSI"+str(isi)+"V"+str(int(vdcc_number))+".mdl"
            new_lines=[line.replace("VDCC_number_presynaptic = 80","VDCC_number_presynaptic = "+str(int(vdcc_number))) for line in template_lines]
            with open(new_loc,'w') as new_file:
                new_file.writelines(new_lines)


### counting simulations

In [10]:
with open("/home/kabir/Project/tripartiteSynapse/results/rec.pkl",'rb') as f:
    rec=pkl.load(f)

for key in rec.keys():
    for sub in rec[key].keys():
        if "I20" in sub:
            print(key, sub, rec[key][sub])

R150control RSI20V84 391
R150control RSI20V70 104
R150control RSI20V98 166
R150control RSI20V94 274
R150control RSI20V138 13
R150control RSI20V115 48
R150control RSI20V78 329
R150control RSI20V128 6
R150control RSI20V124 29
R150control RSI20V120 22
R150ER3x RSI20V84 391
R150ER3x RSI20V70 107
R150ER3x RSI20V98 166
R150ER3x RSI20V94 274
R150ER3x RSI20V138 13
R150ER3x RSI20V115 48
R150ER3x RSI20V78 329
R150ER3x RSI20V128 6
R150ER3x RSI20V124 29
R150ER3x RSI20V120 22
R150ER2x RSI20V84 391
R150ER2x RSI20V70 107
R150ER2x RSI20V98 166
R150ER2x RSI20V94 274
R150ER2x RSI20V138 13
R150ER2x RSI20V115 48
R150ER2x RSI20V78 329
R150ER2x RSI20V128 6
R150ER2x RSI20V124 29
R150ER2x RSI20V120 22
R300ER2x RSI20V84 389
R300ER2x RSI20V70 107
R300ER2x RSI20V98 166
R300ER2x RSI20V94 274
R300ER2x RSI20V138 13
R300ER2x RSI20V115 48
R300ER2x RSI20V78 329
R300ER2x RSI20V128 6
R300ER2x RSI20V124 29
R300ER2x RSI20V120 22
R300ER3x RSI20V84 391
R300ER3x RSI20V70 104
R300ER3x RSI20V98 166
R300ER3x RSI20V94 274
R300ER

# kleak conversion to mcell

In [69]:
#from S Bandara et al 2013
J=0.07*250 #uM/s
J/=1e15/N_avo/vol_er*1e6
print("Ca Flux in ions/sec = ",J)

p=[2.1675959461586873, 6.214988757731395]
kleak=(J-p[1])/p[0]
print("Predicted kleak = ",kleak)

Ca Flux in ions/sec =  411.01111642500007
Predicted kleak =  186.74888573427603


# creating run.sh file

In [8]:
def seed_num(vdcc_num):
    #vdcc_num=int(fname.split('V')[1].replace('.mdl',''))
    if vdcc_num>=110 and vdcc_num<=160:
        return 1000
    elif vdcc_num>=90 and vdcc_num<=100:
        return 2000
    elif vdcc_num>=70 and vdcc_num<=80:
        return 3000
    elif vdcc_num>=40 and vdcc_num<=60:
        return 5000
    else:
        return 10

### create job list along with no. of seeds:

In [14]:
#Murthy 1997 dist

sims=["R150control_clamped","R300ER2x_clamped" ,"R300ER3x_clamped","R150ER3x_clamped" ,"R150ER2x_clamped"]#"stores_blocked",
script_folder="/home/kabir/Project/tripartiteSynapse/mcell"#+'/'+sim_type
print(script_folder)
isi=list(isi_range)
VDCC=vdcc_numbers

print('isi = '+str(isi),'\nvdcc = '+str(VDCC))
fnames=[]
seeds=[]
for (i,v,sim) in product(isi,vdcc_dist.keys(),sims):
    if "RSI"+str(i)+"V"+str(v) not in rec[sim.replace("_clamped","")].keys():
        fnames.append(sim+"/RSI"+str(i)+"V"+str(v)+".mdl")
        seeds.append((1,int(vdcc_dist[v]))) #seed_num(v)
    #print(d/sum(dist.T[1]))
list(zip(fnames,seeds))

/home/kabir/Project/tripartiteSynapse/mcell
isi = [20, 60, 100, 140, 180] 
vdcc = [ 70.  78.  84.  89.  94.  98. 103. 107. 111. 115. 120. 124. 128. 133.
 138. 143. 149. 157.   0.   0.]


[('R150control_clamped/RSI20V89.mdl', (1, 326)),
 ('R300ER2x_clamped/RSI20V89.mdl', (1, 326)),
 ('R300ER3x_clamped/RSI20V89.mdl', (1, 326)),
 ('R150ER3x_clamped/RSI20V89.mdl', (1, 326)),
 ('R150ER2x_clamped/RSI20V89.mdl', (1, 326)),
 ('R150control_clamped/RSI20V103.mdl', (1, 120)),
 ('R300ER2x_clamped/RSI20V103.mdl', (1, 120)),
 ('R300ER3x_clamped/RSI20V103.mdl', (1, 120)),
 ('R150ER3x_clamped/RSI20V103.mdl', (1, 120)),
 ('R150ER2x_clamped/RSI20V103.mdl', (1, 120)),
 ('R150control_clamped/RSI20V107.mdl', (1, 78)),
 ('R300ER2x_clamped/RSI20V107.mdl', (1, 78)),
 ('R300ER3x_clamped/RSI20V107.mdl', (1, 78)),
 ('R150ER3x_clamped/RSI20V107.mdl', (1, 78)),
 ('R150ER2x_clamped/RSI20V107.mdl', (1, 78)),
 ('R150control_clamped/RSI20V111.mdl', (1, 58)),
 ('R300ER2x_clamped/RSI20V111.mdl', (1, 58)),
 ('R300ER3x_clamped/RSI20V111.mdl', (1, 58)),
 ('R150ER3x_clamped/RSI20V111.mdl', (1, 58)),
 ('R150ER2x_clamped/RSI20V111.mdl', (1, 58)),
 ('R150control_clamped/RSI20V133.mdl', (1, 6)),
 ('R300ER2x_cla

In [40]:
#VDCC range

sims=["R150control"]#,"R150ER2x","R300ER2x","R150ER3x","R300ER3x"]#,"stores_blocked" "ryr_old",
#sim_type=sims[2]
script_folder="/home/kabir/Project/tripartiteSynapse/mcell"#+'/'+sim_type
print(script_folder)
isi=[20]
VDCC=list(range(140,200,10))

print('isi = '+str(isi),'\nvdcc = '+str(VDCC))
fnames=[]
seeds=[]
for (i,v,sim) in product(isi,VDCC,sims):
    fnames.append(sim+"_clamped/RSI"+str(i)+"V"+str(v)+".mdl")
    seeds.append((1,1000)) #seed_num(v)
print(fnames,seeds)


/home/kabir/Project/tripartiteSynapse/mcell
isi = [20] 
vdcc = [140, 150, 160, 170, 180, 190]
['R150control_clamped/RSI20V140.mdl', 'R150control_clamped/RSI20V150.mdl', 'R150control_clamped/RSI20V160.mdl', 'R150control_clamped/RSI20V170.mdl', 'R150control_clamped/RSI20V180.mdl', 'R150control_clamped/RSI20V190.mdl'] [(1, 1000), (1, 1000), (1, 1000), (1, 1000), (1, 1000), (1, 1000)]


In [5]:
#ISI range

sims=["R150control","R150ER2x","R300ER2x","R150ER3x","R300ER3x"]
sim_type=sims[0]
script_folder="/home/kabir/Project/tripartiteSynapse/mcell"+'/'+sim_type
print(script_folder)
isi=list(range(20,101,20))
VDCC=[90]
print('isi = '+str(isi),'\nvdcc = '+str(list(VDCC)))
fnames=[]
for (i,v) in product(isi,VDCC):
    fnames.append("RSI"+str(i)+"V"+str(v)+".mdl")
#fnames=["RSI20V90.mdl","RSI30V90.mdl","RSI40V90.mdl"]
print(fnames)
#seed_num=1000

/home/kabir/Project/tripartiteSynapse/mcell/R150control
isi = [20, 40, 60, 80, 100] 
vdcc = [90]
['RSI20V90.mdl', 'RSI40V90.mdl', 'RSI60V90.mdl', 'RSI80V90.mdl', 'RSI100V90.mdl']


In [21]:
#VDCC range

sims=["R150control","R150ER2x","R300ER2x","R150ER3x","R300ER3x"]
sim_type=sims[2]
script_folder="/home/kabir/Project/tripartiteSynapse/mcell"+'/'+sim_type
print(script_folder)
isi=[20]
VDCC=list(range(40,161,20))
print('isi = '+str(isi),'\nvdcc = '+str(list(VDCC)))
fnames=[]
for (i,v) in product(isi,VDCC):
    fnames.append("RSI"+str(i)+"V"+str(v)+".mdl")
print(fnames)

/home/kabir/Project/tripartiteSynapse/mcell/R300ER2x
isi = [20] 
vdcc = [40, 60, 80, 100, 120, 140, 160]
['RSI20V40.mdl', 'RSI20V60.mdl', 'RSI20V80.mdl', 'RSI20V100.mdl', 'RSI20V120.mdl', 'RSI20V140.mdl', 'RSI20V160.mdl']


In [4]:
#3003x

sims=["R150control","R150ER2x","R300ER2x","R150ER3x","R300ER3x"]
sim_type=sims[4]
script_folder="/home/kabir/Project/tripartiteSynapse/mcell"+'/'+sim_type
print(script_folder)
isi=[20]
VDCC=[70]
print('isi = '+str(isi),'\nvdcc = '+str(list(VDCC)))
fnames=[]
for (i,v) in product(isi,VDCC):
    fnames.append("RSI"+str(i)+"V"+str(v)+".mdl")
print(fnames)

/home/kabir/Project/tripartiteSynapse/mcell/R300ER3x
isi = [20] 
vdcc = [70]
['RSI20V70.mdl']


In [5]:
sims=["R150control","R150ER2x","R300ER2x","R150ER3x","R300ER3x"]
script_folder="/home/kabir/Project/tripartiteSynapse/mcell"
fnames=[s+'/RS20p20hz.mdl' for s in sims]
print(fnames)

['R150control/RS20p20hz.mdl', 'R150ER2x/RS20p20hz.mdl', 'R300ER2x/RS20p20hz.mdl', 'R150ER3x/RS20p20hz.mdl', 'R300ER3x/RS20p20hz.mdl']


#### write freq.sh

In [38]:
with open(script_folder+'/freq.sh','w') as f:
    for name in fnames:
            loc=script_folder+'/'+name
            f.write("sed --i 's/^#PBS -J.*/#PBS -J 1-"+str(1000)+"/' pbs.py\n")
            f.write("qsub -N "+name.replace("RS20p20hz.mdl",'').replace(script_folder,'')+" -v I='"+loc.replace('/kabir/Project/','/subhadra/kabir/')+"' pbs.py\n")
            f.write("\n")

#### write run.sh

In [15]:
with open(script_folder+'/run.sh','w') as f:
    for (name,s) in zip(fnames,seeds):
        if s[1]>s[0]:
            loc=script_folder+'/'+name
            #f.write("sleep 5\n")
            f.write("sed --i 's/^#PBS -J.*/#PBS -J "+str(s[0])+"-"+str(s[1])+"/' pbs.py\n")
            f.write("qsub -N "+name.replace("_clamped",'').replace(".mdl",'')+" -v I='"+loc.replace('/kabir/Project/','/subhadra/kabir/')+"' pbs.py\n")
            f.write("\n")

### copy run.sh to cluster

In [16]:
#sync script files to cluster
p=subprocess.call(["rsync","-arhP",script_folder+'/run.sh',"subhadra@192.168.1.244:/home/subhadra/kabir/tripartiteSynapse/mcell/"])

### copy sims folders to cluster

In [27]:
#sync script files to cluster
for s in sims:
    p=subprocess.call(["rsync","-arhP",script_folder+'/'+s,"subhadra@192.168.1.244:/home/subhadra/kabir/tripartiteSynapse/mcell/"])

In [42]:
#sync data back to this system
p=subprocess.call(["rsync","-arhP","subhadra@192.168.1.244:/storage/subhadra/kabir/output/ppf/"+outfolder,"/data/kabir/output/ppf/"])

In [17]:
outfolder='R300ER3x'#'R150control'#RSI40V90'

In [18]:
#sync data back to this system
p=subprocess.call(["rsync","-arhP","subhadra@192.168.1.244:/storage/subhadra/kabir/result/ppf/"+outfolder,"/home/kabir/Project/tripartiteSynapse/results/ppf/"])